In [306]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib, os, fiona
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
from shapely.ops import nearest_points, polygonize, linemerge,unary_union
from shapely.geometry import Point,MultiLineString
from collections import Counter
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import Table, Column, Integer, String, MetaData
from sqlalchemy.sql import select
from sklearn.cluster import DBSCAN
import Outils
#pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# A FAIRE : 1
- "triangle" multiple : TRONROUT0000000032823937
- mauvais split : TRONROUT0000000032848625
- erreur rd pt : TRONROUT0000000032840707
- 2*2 voies voies communales : TRONROUT0000000032848788 (ss les rd points ç amarche bien, ça pourrait etre une solution...)
- triangle foireux : TRONROUT0000000032839017

# IMPORTER LES DONNEES 
> depuis la base de données locale, en ne prenant en compte que les lignes qui intersectent une zone carré au N-O

In [2]:
df=at.import_donnes_base('local_otv')
df2_chaussees=df.loc[df.nature.isin(['Autoroute', 'Quasi-autoroute', 'Route à 2 chaussées'])]

In [3]:
df.to_file(r'D:\temp\otv\test_linearisation\donnees_test.shp')

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


# 1 RONDS POINTS

In [4]:
df_avec_rd_pt,carac_rd_pt,lign_entrant_rdpt=at.identifier_rd_pt(df)
df_lignes=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [5]:
df_avec_rd_pt[['id_ign', 'geom', 'id', 'nature', 'numero', 'importance', 'cl_admin',
       'gestion', 'mise_serv', 'fictif', 'franchisst', 'largeur', 'z_ini',
       'z_fin', 'nb_voies', 'sens', 'etat', 'long_km', 'coment_cpt', 'src_cpt',
       'id_comptag', 'obs_supl', 'ann_pt', 'coment_tmj', 'tmja', 'tmja_final',
       'pc_pl', 'obs_tmja', 'obs_pc_pl', 'type_vdf', 'vts_type', 'vma',
       'veh_km', 'id_cpt1', 'id_cpt2', 'obs_tmj1', 'obs_tmj2', 'id_sect',
       'src_sect', 'autor_sect', 'codau_cat', 'milieu', 'vts_vl_vdf',
       'vts_pl_vdf', 'vts_gest', 'obs_vts', 'vts_osm', 'vts_modif', 'vts_vl_f',
       'vts_pl_f', 'codau', 'codau_cont', 'tmja_cont', 'pc_pl_cont', 'source',
       'target', 'recup', 'attr_modif', 'codevoie_d', 'id_tronc_elem',
       'nb_intrsct_src', 'src_geom', 'nb_intrsct_tgt', 'tgt_geom', 'id_rdpt',
       'nb_rte_rdpt', 'nb_obj_sig_entrant']].to_file(r'D:\temp\otv\test_linearisation\df_avec_rd_pt.shp')

# 2. TRAITEMENTS 
> La base c'est : 
- regrouper les lignes qui se suivent
- trouver les lignes d'arret
- trouver les lignes suivantes
- voir le cas des lignes suivantes : 
    - rd point
    - mm route qui se sépare
    - rien

><br> L'idée c'est ça  : on met la premiere ligne a tester dans une liste, on l'utilise et on stocke les lignes associées dans une autre liste. ensuite on check les lignes suivantes : si elles sont dans un cas où il faut continuer les traitements, on remplace la ligne de départ dans la liste de départ par ces nouvelles lignes, pour lesquelles on applique de nouveau la recherche des lignes qui se suivent. tant que la liste des lignes suivantes est pa vide on reste dans une boucle while

## 2.1 Troncons de base

In [190]:
# ligne a tester : TRONROUT0000000339411609 TRONROUT0000000032869130 TRONROUT0000000032859410
id_lignes='TRONROUT0000000032834954'
#trouver tous les troncons proches qui se suivent
#list_troncon=[id_ligne]+[a for a in at.liste_troncon_base(id_ligne,df_lignes)]
list_troncon=at.liste_complete_tronc_base(id_lignes,df_lignes,[])

## 2.2 lignes d'arret

In [191]:
#trouver les troncons de debut et de fin :ceux pour lesquels on a un nombre de ligne sur target source >2
dico_deb_fin=at.deb_fin_liste_tronc_base(df_lignes, list_troncon)

In [192]:
dico_deb_fin

{0: {'id': 'TRONROUT0000000032834954',
  'type': 'source',
  'num_node': 4018,
  'geom_node': 'POINT(412859.8 6574660.49999999)',
  'voie': 'NC',
  'codevoie': 'NR'},
 1: {'id': 'TRONROUT0000000032834958',
  'type': 'target',
  'num_node': 72644,
  'geom_node': 'POINT(412241.7 6575028.99999999)',
  'voie': 'NC',
  'codevoie': 'NR'}}

In [193]:
ligne='TRONROUT0000000032834954'
for v in dico_deb_fin.values() : 
    if v['id']==ligne :
        voie=v['voie']
        noeud=v['num_node']
        geom_noeud=v['geom_node']
        type_noeud=v['type']
        codevoie=v['codevoie']
        #break
    else : 
        continue

## 2.3 lignes suivantes

In [194]:
#vérifier si une ligne touche un rd point
# aprtir d'une ligne et d'un numero de noeud, on récupère les lignes qui touchent
lignes_adj=df_lignes.loc[((df_lignes['source']==noeud)|(df_lignes['target']==noeud))&(df_lignes.index!=ligne)]

In [195]:
lignes_adj

,geom,id,nature,numero,importance,cl_admin,gestion,mise_serv,fictif,franchisst,largeur,z_ini,z_fin,nb_voies,sens,etat,long_km,coment_cpt,src_cpt,id_comptag,obs_supl,ann_pt,coment_tmj,tmja,tmja_final,...,vts_osm,vts_modif,vts_vl_f,vts_pl_f,codau,codau_cont,tmja_cont,pc_pl_cont,source,target,recup,attr_modif,codevoie_d,id_tronc_elem,nb_intrsct_src,src_geom,nb_intrsct_tgt,tgt_geom,id_rdpt,nb_rte_rdpt,nom_rte_rdpt_entrant,codevoie_rdpt_entrant,nb_obj_sig_entrant,valeur_sens,id_ign_entrant
id_ign,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TRONROUT0000000130604054,"(LINESTRING (412880.7999999996 6574660.599999992, 412871.3999999997 6574660.099999992, 412859.7999999997 6574660.499999993))",215196,Route à 1 chaussée,NC,4,Autre,NC,NR,Non,NC,4.0,11.5,11.7,1.0,Direct,NR,0.021020,estimation,None,None,None,None,/2,1430.0,715.0,...,None,None,80.0,74.0,111+,None,None,None,194,4018,None,None,NR,NaN,3,POINT(412880.8 6574660.59999999),3,POINT(412859.8 6574660.49999999),NaN,NaN,NaN,NaN,NaN,NaN,NaN
TRONROUT0000000130604057,"(LINESTRING (412880.6999999998 6574643.399999994, 412870.5999999996 6574652.399999993, 412859.7999999997 6574660.499999993))",215199,Route à 1 chaussée,NC,4,Autre,NC,NR,Non,NC,4.0,11.5,11.7,1.0,Inverse,NR,0.027028,estimation,None,None,None,None,/2,1430.0,715.0,...,None,None,80.0,74.0,111+,None,None,None,4017,4018,None,None,NR,NaN,4,POINT(412880.7 6574643.39999999),3,POINT(412859.8 6574660.49999999),NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.4 cas particulier 

### 2.4.1 rd pt

In [47]:
#verfier si une ligne rouche un rd point et recup le numero si c'est le cas
check_rdpt, num_rdpt=at.verif_touche_rdpt(lignes_adj)
if check_rdpt : 
    lignes_rdpt, lignes_sortantes=at.recup_lignes_rdpt(carac_rd_pt,num_rdpt,list_troncon,voie,codevoie)

### 2.4.2 mm route qui se sépare

In [196]:
#comparer le nom de voie de la ligne issue de debut_fin avec celui des lignes adjacentes
if len(set(lignes_adj.source.tolist()+lignes_adj.target.tolist())) : #cas d'une ligne qui separe pour se reconnecter ensuite
    print(lignes_adj.index.tolist())
if (voie==lignes_adj.numero).all() and voie!='NC' : 
    print('if')
    ligne_mm_voie=lignes_adj.index.tolist()
    list_troncon+=ligne_mm_voie
    lignes_suivantes_mm_nom=ligne_mm_voie #on recommence les traitements avec les lignes prises
elif voie=='NC' and codevoie!='NR' and (codevoie==lignes_adj.codevoie_d).all(): 
    print('elif')
    ligne_mm_voie=lignes_adj.index.tolist()
    list_troncon+=ligne_mm_voie
    lignes_suivantes_mm_nom=ligne_mm_voie
else : # cas des nc / nr qui se sépare : ont réflechi en angle et longueurs eéquivalente, avec si besoin comparaiosn des lignes qui touvhent
    #si une des lignes qui se separent fait partie d'un rd point on passe
    if (lignes_adj.id_rdpt>0).any()==1  : 
        print('passer')
    lgn_cote_1, lgn_cote_2, angle_1,angle_2,angle_3=at.angle_3_lignes('TRONROUT0000000039535695', lignes_adj, noeud,geom_noeud, type_noeud, df_lignes)
    lignes_adj_tot=lignes_adj.copy()
    lignes_adj_tot['tronc_supp']=lignes_adj_tot.apply(lambda x : at.liste_complete_tronc_base(x.name,df_lignes,[id_lignes]), axis=1)
    geom_l1, lg_l1=at.fusion_ligne_calc_lg(df_lignes.loc[lignes_adj_tot.loc[lgn_cote_1.name].tronc_supp])
    geom_l2, lg_l2=at.fusion_ligne_calc_lg(df_lignes.loc[lignes_adj_tot.loc[lgn_cote_2.name].tronc_supp])
    geom_ldep, lg_ldep=at.fusion_ligne_calc_lg(df_lignes.loc[list_troncon])
    tt_tronc=[x for y in lignes_adj_tot.tronc_supp.tolist() for x in y ]
    noeuds_fin=[k for k,v in Counter(df_lignes.loc[tt_tronc].source.tolist()+df_lignes.loc[tt_tronc].target.tolist()).items() if v==1]
    if (min(lg_l1, lg_l2) / max(lg_l1, lg_l2)) > 0.66 : # soit les lignes qui se separent ont a peu pres la mm longueur et un angle entre elle faible et une ligne les touchent toute les deux (dans ce cas la ligne de depart est la grandee ligne qui arrive)
        lignes_jointure=df_lignes.loc[(df_lignes.source.isin(noeuds_fin)) & (df_lignes.target.isin(noeuds_fin))]
        print('dehors')
        if not lignes_jointure.empty : 
            print('dedans')
            if angle_3<60 :
                print(tt_tronc)
            else : 
                print('pas angle')
        else : 
            print('exit')
    elif ((min(lg_l1, lg_ldep) / max(lg_l1, lg_ldep)) > 0.66) and angle_1<75 : #soit on est dans le cas où la ligne dedépart est une des petites lignes pres du rd point
        print('ok avec 1')
    elif ((min(lg_l2, lg_ldep) / max(lg_l2, lg_ldep)) > 0.66) and angle_2<75 :
        print('ok avec 2')

['TRONROUT0000000130604054', 'TRONROUT0000000130604057']
dehors
exit


In [187]:
lignes_adj.nature.isin(['Autoroute', 'Quasi-autoroute', 'Route à 2 chaussées']).any()

True

In [189]:
at.recup_route_split('TRONROUT0000000032830740',list_troncon,voie,codevoie, lignes_adj,noeud,geom_noeud, type_noeud, df_lignes)

[]

### 2.4.3 trouver les routes coinces entre 2 voies qui se sépare -<|
 ce sont des lignes qui  dont : 
- si une des lignes adjacente à le mm nom de voie
- on prend l'autre si elle a un sens direct ou inverse
- on recherche son source ou target different du num_node de dico_deb_fin
- on cherche les lignes qui touche ce noeud, qui ont le mme nom de voie et un sens direct ou inverse
- on recupere cette ligne, et le num de noeud differnts de celui de la 1ere ligne qui bifurque
- si ce num de noeud correspond à la ligne adj source ou target, alors cette ligne adj source ou target doit être ajoutée à la liste

In [923]:
ligne_depart='TRONROUT0000000318255826'
if voie != 'NC' : 
    lgn_cote_1=lignes_adj.loc[(lignes_adj['numero']!=voie)]  
    if lgn_cote_1.empty : 
        print([]) 
    noeud_centre = lgn_cote_1['source'].values[0] if lgn_cote_1['source'].values[0] != noeud else lgn_cote_1['target'].values[0]
    lgn_suiv=lignes_adj.loc[~lignes_adj.index.isin(lgn_cote_1.index.tolist())]
    if lgn_suiv.empty : 
        print([]) 
    noeud_suiv=lgn_suiv['source'].values[0] if lgn_suiv['source'].values[0] != noeud else lgn_suiv['target'].values[0]

    #dans le cas ou plusieurs ligne a la suite forme le cote du triangle que l'on veut récupérer on veut connaitre le nb de ligne qui touchent le noeud_suiv
    if lgn_suiv['source'].values[0] != noeud : 
        noeud_suiv=lgn_suiv['source'].values[0]
        nb_intersect_noeud_suiv=df_lignes.loc[lgn_suiv.index.values[0]].nb_intrsct_src
    else : 
        noeud_suiv=lgn_suiv['target'].values[0]
        nb_intersect_noeud_suiv=df_lignes.loc[lgn_suiv.index.values[0]].nb_intrsct_tgt

    if  nb_intersect_noeud_suiv==2 : 
        lgn_suiv=df_lignes.loc[at.liste_complete_tronc_base(lgn_suiv.index.values[0],df_lignes,[])] 
        noeud_ss_suite=Counter(lgn_suiv.source.tolist()+lgn_suiv.target.tolist())
        noeud_suiv=[k for k,v in noeud_ss_suite.items() if v==1 and v!=noeud][0]

    id_rattache=lgn_suiv.index.tolist()
    if noeud_centre==noeud_suiv : #ça veut dire une seule et mm lign qui fait 2 cote du triangle, dc on peu de suite garder la ligne suiv car 1 seule route non coupe l'intersec
        print (id_rattache)
    else : # on cherche si une ligne touche ces 2 noeuds, a le mme nom de voie que la ligne de cote  et un sens direct ou inverse
        ligne_a_rattacher=df_lignes.loc[((df_lignes['source']==noeud_centre) | (df_lignes['target']==noeud_centre)) & 
                  (df_lignes['numero']==lgn_cote_1['numero'].values[0]) & (~df_lignes.index.isin(lgn_cote_1.index.tolist())) & 
                  ((df_lignes['source']==noeud_suiv) | (df_lignes['target']==noeud_suiv))]
        if not ligne_a_rattacher.empty : 
            print( id_rattache)
        else :
            print([]) 

['TRONROUT0000000032839436', 'TRONROUT0000000318255825']


In [925]:
at.recup_triangle(ligne,voie, codevoie,lignes_adj, noeud,geom_noeud, type_noeud, df_lignes)

['TRONROUT0000000032839436', 'TRONROUT0000000318255825']

## 2.5 route à 2 chaussées
L'idée c'est si la ligne est une route à 2 chaussée : 
- on prend l'ensemble du troncon elementaire
- on agerge les lignes
- on recupere le centre
- on recupere la longueur
- on pourrait chercher à comparer les bouding box

In [69]:
#dans le cas où les longeuers ne coincident pas
#recuperer la ligne proche
ligne_proche, ligne_filtres, longueur_base=at.trouver_chaussees_separee(list_troncon, df_avec_rd_pt)

In [76]:
at.gestion_voie_2_chaussee(list_troncon, df_avec_rd_pt, 'TRONROUT0000000032831888')[0]

['TRONROUT0000000032831889',
 'TRONROUT0000000329876240',
 'TRONROUT0000000329876239',
 'TRONROUT0000000032831831',
 'TRONROUT0000000032831834',
 'TRONROUT0000000032831830']

# Tests de deroulement global

In [43]:
at.lignes_troncon_elem(df_avec_rd_pt,carac_rd_pt,'TRONROUT0000000032840667')

['TRONROUT0000000032840703',
 'TRONROUT0000000032840702',
 'TRONROUT0000000032840707',
 'TRONROUT0000000032840708',
 'TRONROUT0000000032840704',
 'TRONROUT0000000032840667',
 'TRONROUT0000000032840670']

# 3. Boucler sur 'ensembl des lignes
On va faire tourner les traitements sur l'ensemble des lignes, en partant de la liste des id_ign, pour chaque ligne : 
- si la ligne ne remplit pas les critères d'exclusion on continu, sinon on passe à la suivante
- fonction de recherche des lignes associées
- df des association
- filtre de la liste des id_ign de base

In [6]:
#traiter les bretelles en 1er
list_id_ign=df_avec_rd_pt.loc[df_avec_rd_pt['nature']=='Bretelle'].id_ign.tolist()+df_avec_rd_pt.loc[df_avec_rd_pt['nature']!='Bretelle'].id_ign.tolist()

In [9]:
df_affectation, dico_erreur, lignes_traitees, lignes_non_traitees=at.regrouper_troncon(
    list_id_ign, df_avec_rd_pt, carac_rd_pt,df2_chaussees)

0 2019-09-23 11:08:28.370462 nb lignes traitees : 0
500 2019-09-23 11:10:02.622256 nb lignes traitees : 2667
1000 2019-09-23 11:11:34.375514 nb lignes traitees : 5027
1500 2019-09-23 11:12:47.258431 nb lignes traitees : 6583
2000 2019-09-23 11:13:59.741345 nb lignes traitees : 8040
2500 2019-09-23 11:14:51.338637 nb lignes traitees : 9166
3000 2019-09-23 11:15:37.464634 nb lignes traitees : 10058
3500 2019-09-23 11:16:12.378306 nb lignes traitees : 10757
4000 2019-09-23 11:16:44.730115 nb lignes traitees : 11365
4500 2019-09-23 11:17:22.783674 nb lignes traitees : 11924
5000 2019-09-23 11:17:55.322935 nb lignes traitees : 12372
5500 2019-09-23 11:18:25.991196 nb lignes traitees : 12869
6000 2019-09-23 11:18:50.634470 nb lignes traitees : 13191
6500 2019-09-23 11:19:13.556862 nb lignes traitees : 13448
7000 2019-09-23 11:19:40.352704 nb lignes traitees : 13689
7500 2019-09-23 11:20:03.623159 nb lignes traitees : 13943
8000 2019-09-23 11:20:23.218769 nb lignes traitees : 14149
8500 2019-

In [10]:
df_avec_rd_pt.merge(df_affectation, left_on='id_ign',right_on='id')[[
    'id_ign','numero','codevoie_d','geom','idtronc']].to_file(r'D:\temp\otv\test_linearisation\test_13.shp')

# 4. Creer graph
- en premier on traite les cas particuliers  : beretelles, petits troncons
- on cerche a creer une df avec tronc_elem|tronc_ign|src_elem|tgt_elem
- il faudrait un filtre sur les petits troncon entre 2 grands et les associer aux grands
- il faudrait regrouper les bretelles aussi

In [11]:
#calculer les longeurs des tronc_elem et avoir aussi les id qui les composent
tronc_elem_geom=df_avec_rd_pt.merge(df_affectation, left_on='id_ign',right_on='id')[['geom','idtronc']].dissolve(by='idtronc')
tronc_elem_geom['long']=tronc_elem_geom.geom.length
tronc_elem_synth=tronc_elem_geom.merge(df_affectation.groupby('idtronc').agg({'id' : lambda x : tuple(set(x))}), left_index=True, right_index=True)

## 4.1 Bretelles
- trouver les troncons eleentaires qui contiennent des lignes de natures Bretelles
- passer dans Bdd pour utiliser clusterdbscan sur des lignes
- récupérer une df de bretelles avec une agreg posible
- A FAIRE : CREER UNE TABLE DE CORRESPONDANCE ENTRE ID_TRON_ELEM et ID_BRET

In [22]:
#select tronc_elem bretelles
list_id_bretelle=df_lignes.loc[df_lignes['nature']=='Bretelle'].index.tolist()
tronc_bretelle=tronc_elem_synth.loc[tronc_elem_synth.apply(lambda x : any([a for a in x['id'] if a in list_id_bretelle]), axis=1)].copy()
#geom en wkt pour tranfert dans bdd
tronc_bretelle_wkt=tronc_bretelle.reset_index()[['idtronc','geom']].copy()
tronc_bretelle_wkt['geom']=tronc_bretelle_wkt.apply(lambda x : x['geom'].wkt, axis=1)
#on bascule en sql pour utiliser le cluster de postgres qui est plus puissant car il gere tout type de geom et donc la distance entre les geoms devient plus precise
#cree la table qui va accueillir les donnees de bretelles
metadata = MetaData()
bretel_table=Table(
    'bretelle',metadata,
    Column('idtronc', Integer, primary_key=True),
    Column('geom', Geometry()))
requete = "SELECT idtronc, ST_ClusterDBSCAN(geom, eps := 50, minpoints := 2) over () AS cid FROM public.bretelle"

In [23]:
with ct.ConnexionBdd('local_otv') as c :
    bretel_table.create(c.engine)
    c.sqlAlchemyConn.execute(bretel_table.insert(), tronc_bretelle_wkt.to_dict('records'))
    df_cluster=pd.read_sql_query(requete, c.sqlAlchemyConn)

In [28]:
df_cluster.loc[df_cluster.cid.isna()]

,idtronc,cid
13,14,NaN
16,17,NaN
35,47,NaN
48,77,NaN
63,116,NaN


## 4.2 petit troncon

In [390]:
petit_tronc=tronc_elem_synth.loc[tronc_elem_synth['long']<=50]# 467 troncons
list_petit_tronc=[b for a in petit_tronc.id.tolist() for b in a]#list des id_ign present dans les petits troncons
#trouver la liste des autres troncons qui les touchent avec les carac en terme d'angles, numeros, codevoie_d
def tronc_tch(ids, df_lignes) : 
    """
    obtenir une df des troncon touches par une ligne avec l'angle entre les lignes, le numero, le codevoie_d
    en entree : 
        ids : tuple de string de id_ign
        df_lignes : données issues de identifier_rd_pt() avec id_ign en index
    en sortie : 
        df_tronc_tch : df avec les attributs precites
    """
    if len(ids)==1 : #une seule lignes dans le troncon
        df_ids=df_lignes.loc[ids].copy()
        noeuds = [df_ids.source,df_ids.target] 
        list_ids_tch= df_lignes.loc[((df_lignes.target.isin(noeuds)) | (df_lignes.source.isin(noeuds))) & 
                (~df_lignes.index.isin(ids))].index.tolist()
        list_carac_tch=[(a,b) for a,b in zip(list_ids_tch,[('source',df_lignes.loc[a].source) if df_lignes.loc[a].source in noeuds else 
                      ('target',df_lignes.loc[a].target) for a in list_ids_tch])]
        list_carac_fin=[(a[0][0],a[0][1][0],a[0][1][1], a[1]) for a in zip(list_carac_tch,['source' if b[1]==df_lignes.loc[ids].source else 'target' 
                                       for a,b in list_carac_tch])]
        df_tronc_tch=pd.DataFrame.from_records(list_carac_fin, columns=['id_ign', 'type_noeud_lgn', 'id_noeud_lgn', 'type_noeud_src'])
        #calcul des coordonnées des points 
        df_tronc_tch['coord_lgn_base']=df_tronc_tch.apply(lambda x : [coord for coord in df_lignes.loc[ids].geom[0].coords][1] if 
            x['type_noeud_src']=='source' else [coord for coord in df_lignes.loc[ids].geom[0].coords][-2],axis=1)
        df_tronc_tch['coord_lgn_comp']=df_tronc_tch.apply(lambda x : [coord for coord in df_lignes.loc[x['id_ign']].geom[0].coords][1] if 
            x['type_noeud_lgn']=='source' else [coord for coord in df_lignes.loc[x['id_ign']].geom[0].coords][-2],axis=1)
        df_tronc_tch['coord_noued_centr']=df_tronc_tch.apply(lambda x : [coord for coord in df_lignes.loc[x['id_ign']].geom[0].coords][0] if x['type_noeud_lgn']=='source' 
            else [coord for coord in df_lignes.loc[x['id_ign']].geom[0].coords][-1],axis=1)
        #angle
        df_tronc_tch['angle']=df_tronc_tch.apply(lambda x : Outils.angle_entre_2_ligne(x['coord_noued_centr'],x['coord_lgn_comp'], x['coord_lgn_base']),axis=1)
        df_tronc_tch=df_tronc_tch.merge(df_lignes[['numero','codevoie_d']], left_on='id_ign', right_index=True)
    else : 
        df_ids=df_lignes.loc[list(ids)].copy()
        noeuds=[k for k,v in Counter(df_ids.source.tolist()+df_ids.target.tolist()).items() if v==1] #liste des noeuds uniques
        geom_noeuds=[loads(k).coords[0] for k,v in Counter(df_ids.src_geom.tolist()+df_ids.tgt_geom.tolist()).items() if v==1] #liste des geoms uniques
        geom_ligne=linemerge(unary_union(df_ids.geom.tolist())) #agregation des lignes
        pt_source=tuple([round(a,8) for a in geom_ligne.coords[0]]) #calculdes coordonées deb et fin de la lignes agregee
        pt_target=tuple([round(a,8) for a in geom_ligne.coords[-1]])
        df_noeud=pd.DataFrame(zip(noeuds,geom_noeuds), columns=['id_noeud', 'geom_noeud'])
        df_noeud['type_noeud']=df_noeud.apply(lambda x : 'target' if x['geom_noeud']==pt_target else 'source',axis=1) #affectation du type de noeud
        df_noeud.set_index('id_noeud',inplace=True)
        list_ids_tch= df_lignes.loc[((df_lignes.target.isin(noeuds)) | (df_lignes.source.isin(noeuds))) & 
                        (~df_lignes.index.isin(ids))].index.tolist()
        list_carac_tch=[(a,b) for a,b in zip(list_ids_tch,[('source',df_lignes.loc[a].source) if df_lignes.loc[a].source in noeuds else 
                      ('target',df_lignes.loc[a].target) for a in list_ids_tch])]
        list_carac_fin=[(a[0][0],a[0][1][0],a[0][1][1], a[1]) for a in zip(list_carac_tch,['source' if df_noeud.loc[b[1]].type_noeud=='source' else 'target' 
                                       for a,b in list_carac_tch])]
        df_tronc_tch=pd.DataFrame.from_records(list_carac_fin, columns=['id_ign', 'type_noeud_lgn', 'id_noeud_lgn', 'type_noeud_src'])
        #calcul des coordonnées des points 
        df_tronc_tch['coord_lgn_base']=df_tronc_tch.apply(lambda x : [coord for coord in geom_ligne.coords][1] if 
                    x['type_noeud_src']=='source' else [coord for coord in geom_ligne.coords][-2],axis=1)
        df_tronc_tch['coord_lgn_comp']=df_tronc_tch.apply(lambda x : [coord for coord in df_lignes.loc[x['id_ign']].geom[0].coords][1] if 
                    x['type_noeud_lgn']=='source' else [coord for coord in df_lignes.loc[x['id_ign']].geom[0].coords][-2],axis=1)
        df_tronc_tch['coord_noued_centr']=df_tronc_tch.apply(lambda x : [coord for coord in df_lignes.loc[x['id_ign']].geom[0].coords][0] if x['type_noeud_lgn']=='source' 
            else [coord for coord in df_lignes.loc[x['id_ign']].geom[0].coords][-1],axis=1)
        #angle
        df_tronc_tch['angle']=df_tronc_tch.apply(lambda x : Outils.angle_entre_2_ligne(x['coord_noued_centr'],x['coord_lgn_comp'], x['coord_lgn_base']),axis=1)
        df_tronc_tch=df_tronc_tch.merge(df_lignes[['numero','codevoie_d']], left_on='id_ign', right_index=True)
    return df_tronc_tch

def corresp_petit_tronc(ids,df_lignes,df_affectation,list_petit_tronc) : 
    """
    Obtention du tronc elem auquel rattache un petit tronon, sinon False
    en entree : 
        ids : tuple de string de id_ign
        df_lignes : données issues de identifier_rd_pt() avec id_ign en index
        df_affectation : df de correspondance id_ign - tronc elem, issue de regrouper_troncon()
    en sortie : 
        tronc_elem_ref : integer>0 si corresp, sinon -99
    """
    try : 
        df_tronc_tch=tronc_tch(ids, df_lignes)
    except NotImplementedError : #si sheply bug c'est un cas trop complexe et on laisse tomber
        return -99
    except ValueError : #erreur cree par un rd point foireux
        return -99
    #filtrer les lignes qui correspondent à des petits tronc_elem
    df_tronc_tch=df_tronc_tch.loc[~df_tronc_tch.id_ign.isin(list_petit_tronc)].copy()
    id_ign_ref=df_tronc_tch.loc[df_tronc_tch['angle'].idxmax()].id_ign if 160<df_tronc_tch.angle.max()<200 else False
    #puis trouver le tronc_elem correspondant
    tronc_elem_ref=df_affectation.loc[df_affectation['id']==id_ign_ref].idtronc.values[0] if id_ign_ref else -99
    return tronc_elem_ref

In [392]:
petit_tronc['synt']=petit_tronc.apply(lambda x : corresp_petit_tronc(x['id'],df_lignes,df_affectation,list_petit_tronc), axis=1)

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [391]:
corresp_petit_tronc(('TRONROUT0000000032849817',),df_lignes,df_affectation,list_petit_tronc)

In [394]:
petit_tronc.loc[petit_tronc['synt']==-99]

,geom,long,id,synt
idtronc,,,,
272,"(LINESTRING (392496.6999999997 6575828.099999992, 392503.0999999996 6575830.199999991, 392508.7999999997 6575833.099999992, 392512.2999999996 6575837.299999992, 392514.7999999997 6575843.499999993...",48.666313,"(TRONROUT0000000032833324, TRONROUT0000000206971139)",-99
453,"LINESTRING (389866.1999999997 6580537.399999993, 389869.7999999996 6580529.399999992, 389876.2999999997 6580517.299999992, 389884.0999999997 6580506.499999993)",35.830203,"(TRONROUT0000000032828011,)",-99
653,"(LINESTRING (379834.2999999996 6572555.599999992, 379829.3999999997 6572560.299999991, 379826.6999999997 6572563.099999993), LINESTRING (379877.6999999997 6572592.099999993, 379881.6999999997 6572...",19.803024,"(TRONROUT0000000115741137, TRONROUT0000000032843251)",-99
767,"LINESTRING (379516.0999999997 6570913.299999991, 379525.2999999996 6570912.199999993, 379535.9999999997 6570909.399999992)",20.325817,"(TRONROUT0000000112535615,)",-99
875,"LINESTRING (392707.8999999996 6577544.399999992, 392696.7999999997 6577576.599999993)",34.059507,"(TRONROUT0000000032830833,)",-99
1170,"LINESTRING (389413.3999999997 6574867.299999991, 389416.6999999996 6574857.999999992)",9.868131,"(TRONROUT0000000032835474,)",-99
1211,"LINESTRING (390184.9999999996 6577389.799999991, 390191.6999999997 6577388.199999993)",6.888396,"(TRONROUT0000000039536308,)",-99
1231,"LINESTRING (351524.7999999996 6577447.699999994, 351530.6999999995 6577449.49999999, 351534.8999999996 6577451.899999992, 351537.4999999997 6577454.699999993)",14.826817,"(TRONROUT0000000249470901,)",-99
1554,"LINESTRING (375801.5999999996 6572054.199999993, 375778.2999999996 6572042.499999993)",26.072591,"(TRONROUT0000000032846708,)",-99


## 4.3 sources et target regroupes

In [28]:
def creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation ):
    """
    associer des tronc elem a des noeud représentatif pour créer un graph
    en entree : 
        df_lignes df des lignes bdtopo avec rd point et index=id_ign
        ligne_base : df des lignes constituant le tronc elem. issus de df_lignes
        df_affectation : df de correspondance id_ign idtronc_elem, issu de regrouper_troncon
    en sortie : 
        noeuds_fin : df avec index = noeud_unique (int) et idtronc)=tuple des idtroncelem qui touchent ce noeud
        coresp_te_noeud : toute les correspondances noeud-lignes
        noeud_uniq : list edes noeud de fin de tronc elem
    """
    noeud_uniq=[k for k,v in Counter(ligne_base.source.tolist()+ligne_base.target.tolist()).items() if v==1]
    df_noeud_unique=pd.DataFrame(noeud_uniq, columns=['noeud_uniq'])
    lgn_tch_noeud_uniq=df_lignes.loc[((df_lignes['source'].isin(noeud_uniq)) | (df_lignes['target'].isin(noeud_uniq))) & (~df_lignes.index.isin(ligne_base.index.tolist()))].copy()
    # trouver les correspondance avec doublons de noeud possible et une ligne par corresp
    coresp_ligne_noeud=pd.concat(
        [lgn_tch_noeud_uniq.reset_index().merge(df_noeud_unique, left_on='source', right_on='noeud_uniq')[['id_ign','noeud_uniq']],
         lgn_tch_noeud_uniq.reset_index().merge(df_noeud_unique, left_on='target', right_on='noeud_uniq')[['id_ign','noeud_uniq']]],axis=0)
    #rapatriementdes ifos de longueur du tronc_elem
    coresp_te_noeud=coresp_ligne_noeud.merge(df_affectation, left_on='id_ign', right_on='id')[['id_ign','noeud_uniq','idtronc']].merge(tronc_elem_synth[
        ['long']],left_on='idtronc', right_index=True).rename(columns={'long' : 'long_te'})
    noeuds_fin=coresp_te_noeud.sort_values('noeud_uniq').groupby('noeud_uniq').agg({'idtronc':lambda x : tuple(set(x))})
    noeuds_fin=noeuds_fin.drop_duplicates('idtronc')

    # pour info si plus de 2 boeuds uniques alors il faut filtrer les non représentatif
    if len(noeuds_fin)>2 :
        coresp_te_noeud_filtre_lg=coresp_te_noeud.loc[coresp_te_noeud['long_te']>50] #on ne garde pas les tout petit tronc
        coresp_te_noeud_grp=coresp_te_noeud_filtre_lg.sort_values('noeud_uniq').groupby('noeud_uniq').agg(
            {'idtronc':lambda x : tuple(set(x)),'long_te':lambda x : tuple(set(x))})
        noeuds_fin=coresp_te_noeud_grp.loc[~coresp_te_noeud_grp.apply(lambda x : len(x['idtronc'])==1 ,axis=1)].copy()
        return noeuds_fin,coresp_te_noeud_grp, noeud_uniq # on pourrait creer une erreur pour ca
    return noeuds_fin,coresp_te_noeud, noeud_uniq

In [12]:
def tronc_elem_rdpt(num_rdpt, df_affectation,df_lignes):
    """
    Trouver les tronc_elem qui se rejoigne à un rd point, identifier par un noeud
    en entree : 
       num_rdpt : integer,  id_rdpt de carac_rdpt
       df_affectation : df de correspondance id_ign idtronc_elem, issu de regrouper_troncon
       df_lignes df des lignes bdtopo avec rd point et index=id_ign
    en sortie : 
        df_rdpt_tronc_elem : df avec en index le noeud caracteristique du rd et en data un tuple des tronc_elem arrivant
    """
    tronc_elem_entrant=df_affectation.loc[(df_affectation.id.isin(df_lignes.loc[list(carac_rd_pt.loc[num_rdpt].id_ign_entrant)].index.tolist())) 
                                              & (df_affectation['idtronc']!=num_tronc)].copy()#trouver les tronc_elem qui touchent
    #affectation arbitraire du rd poit à un noeud le composant 
    noeud_uniq_rdpt=df_lignes.loc[carac_rd_pt.loc[num_rdpt]['id_ign_rdpt'][0]]['source']
    tronc_elem_entrant['noeud_uniq']=noeud_uniq_rdpt
    #creer une df de mm structure qu noeuds_fin
    df_rdpt_tronc_elem=tronc_elem_entrant.groupby('noeud_uniq').agg({'idtronc':lambda x : tuple(set(x))})
    return df_rdpt_tronc_elem

In [67]:
def tronc_elem_rd_pt_global (nb_rdpt, ligne_base,num_rdpt, df_affectation,df_lignes) : 
    """
    adaptation de la fonction tronc_elem_rdpt selon le nombre de rdpt du tronc_elem
    en entree : 
        nb_rdpt : integer : nombre de rdpt significatifs
        ligne_base : df des lignes constituant le tronc elem. issus de df_lignes
        num_rdpt : list d'integer,  id_rdpt de carac_rdpt
        df_affectation : df de correspondance id_ign idtronc_elem, issu de regrouper_troncon
        df_lignes df des lignes bdtopo avec rd point et index=id_ign
    en sortie : 
        
    """
    if nb_rdpt==1: # dans ce cas il y a un rdpt qui ferme le troncon d'un coté. On commence par le coté ss rdpt
        noeuds_fin=creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation )[0]
        #puis le cote avec rd_pt : on cherche les troncs elem qui arriven
        df_rdpt_tronc_elem=tronc_elem_rdpt(num_rdpt[0], df_affectation,df_lignes)
        #agreg
        noeuds_fin=pd.concat([noeuds_fin, df_rdpt_tronc_elem], axis=0, sort=False)
    elif nb_rdpt==2 : # adns ce cas il faut trouver les correspondances pour chaque rd pt
        corresp1=tronc_elem_rdpt(num_rdpt[0], df_affectation,df_lignes)
        corresp2=tronc_elem_rdpt(num_rdpt[1], df_affectation,df_lignes)
        noeuds_fin=pd.concat([corresp1, corresp2],axis=0)
    return noeuds_fin

In [95]:
# à partir de df_affectation, je trouve les noeud qui ne sont présent qu'une fois
#regroupement des lignes par idtronc et creation des futurs attributs de src et tgt
group_idtronc=df_affectation.groupby('idtronc').agg({'id':lambda x : tuple(x)})
group_idtronc['src_elem']=-1
group_idtronc['tgt_elem']=-1
#pour une ligne : retrouver les lignes de base

for i,num_tronc in enumerate(df_affectation.idtronc.unique()) :
    if i%1000==0 : 
        print(f'nb lignes traitees {i}, heure : {datetime.now}, tronc :{num_tronc}')
    #num_tronc=68 #396 #3402 #206
    ligne_base=df_lignes.loc[list(group_idtronc.loc[num_tronc].id)]

    # Le pb c'est de gérer les rdpt qui termine un tronc elem. 
    #dc 1. combien de rd point dans le tronc eleme
    nb_rdpt=ligne_base.id_rdpt.nunique()

    #2. Gérer les différents cas du nb de rdpt 
    #si 0 : on va chercher les noeud qui n'apparaissent qu'une fois : ce son des fin de troncon elem
    if nb_rdpt==0 :
        noeuds_fin,synthese_croisement, noeud_uniq=creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation )
    else :  #on attaque les cas en fonction du nombre de rdpt   
        list_rdpt=ligne_base.loc[ligne_base['id_rdpt'].notna()].id_rdpt.unique()
    #si 1 : ce rd point est-il en fin de trocnon ou non. C'est tarduit par le nb de route différentes qui intersectenet les rd point (attribut de carac_rd_pt)
        if nb_rdpt==1 : 
            nb_voie_entrante=carac_rd_pt.loc[num_rdpt].nb_rte_rdpt
            if nb_voie_entrante==1 : #ds ce cas c'est comme si pas de rdpt
                noeuds_fin=creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation )[0]
            else : 
                noeuds_fin=tronc_elem_rd_pt_global(nb_rdpt, ligne_base,list_rdpt, df_affectation,df_lignes)         
        else : #si plusieurs rd pt
            #il faut connaitre le nb de rd pt avec plusieurs voie entrante. sinon le max est 2 si plus que 2 alors erreur
            tt_rdpt=carac_rd_pt.loc[list_rdpt] # tous les rd pt du tronc elem
            rd_pt_multi_voie=tt_rdpt.loc[tt_rdpt['nb_rte_rdpt']>1].index.tolist() #que les multi voies
            if len(rd_pt_multi_voie)==0 : # c'est le cas ss rdpt
                noeuds_fin=creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation )[0] 
            elif len(rd_pt_multi_voie)==1 : # si 1 seul on revient sur le cas du dessus,
                noeuds_fin=tronc_elem_rd_pt_global(1, ligne_base,rd_pt_multi_voie, df_affectation,df_lignes)
            elif len(rd_pt_multi_voie)==2 : # adns ce cas il faut trouver les correspondances pour chaque rd pt
                noeuds_fin=tronc_elem_rd_pt_global(2, ligne_base,rd_pt_multi_voie, df_affectation,df_lignes)
            else : #alors c'est une erreur
                print('erreur nb de rd point significatif, revoir le sectionnement')

    #mettre à jour la df créée au départ : 
    dico_attribut={0:'src_elem', 1:'tgt_elem'}
    for i in range(len(noeuds_fin)) : 
        if i >= 2 : 
            print(f'erreur sur troncon {num_tronc}, df noeud :  {noeuds_fin}')
        group_idtronc.loc[num_tronc, dico_attribut[i]]=noeuds_fin.index.values[i] #mise à jour des valuers de pour le troncon de base
        group_idtronc.loc[num_tronc, dico_attribut[i]]=noeuds_fin.index.values[i]
    for num_tronc_sup in [x for y in noeuds_fin.idtronc.tolist() for x in y] : #mise à jour des valeusr pour les autres troncons
        for i in range(len(noeuds_fin)) :
            if group_idtronc.loc[num_tronc_sup][dico_attribut[0]]==-1 : 
                group_idtronc.loc[num_tronc_sup, dico_attribut[0]]=noeuds_fin.loc[noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)].index.values[0]
                break
            elif group_idtronc.loc[num_tronc_sup][dico_attribut[1]]==-1 :
                group_idtronc.loc[num_tronc_sup, dico_attribut[1]]=noeuds_fin.loc[noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)].index.values[0]
                break
            else : 
                print(f'{num_tronc_sup} deja renseigne' )

nb lignes traitees 0, heure : <built-in method now of type object at 0x00007FFEAC903D60>, tronc :0
5 deja renseigne
5 deja renseigne
8 deja renseigne
8 deja renseigne
erreur sur troncon 6337, df noeud :                   idtronc                                  long_te
noeud_uniq                                                       
66001       (11016, 500)  (353.78757309698466, 5467.885551698945)
72616       (10875, 500)   (5467.885551698945, 309.6104307447416)
73237       (1892, 8853)   (3097.05124851706, 423.14520467961364)
83740       (2354, 1892)    (3097.05124851706, 361.3033189049768)


KeyError: 2

In [82]:
noeuds_fin

,idtronc
noeud_uniq,
72297,"(4504, 3713)"
69869,"(2741, 3005)"


In [47]:
noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)

noeud_uniq
39594     True
52060    False
dtype: bool

In [20]:
synthese_croisement.loc[~synthese_croisement.apply(lambda x : len(x['idtronc'])==1 ,axis=1)].copy()

,idtronc,long_te
noeud_uniq,,
9466,"(938, 1563, 12422)","(445.15375219309715, 1054.161900553276, 431.0587952118286)"


In [328]:
#mettre à jour la df créée au départ : 
group_idtronc.loc[num_tronc, 'src_elem']=noeuds_fin.index.values[0] #mise à jour des valuers de pour le troncon de base
group_idtronc.loc[num_tronc, 'tgt_elem']=noeuds_fin.index.values[1]
for num_tronc_sup in [x for y in noeuds_fin.idtronc.tolist() for x in y] : #mise à jour des valeusr pour les autres troncons
    if group_idtronc.loc[num_tronc_sup].src_elem==-1 : 
        group_idtronc.loc[num_tronc_sup, 'src_elem']=noeuds_fin.loc[noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)].index.values[0]
    elif group_idtronc.loc[num_tronc_sup].tgt_elem==-1 :
        group_idtronc.loc[num_tronc_sup, 'tgt_elem']=noeuds_fin.loc[noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)].index.values[0]
    else : 
        print(f'{num_tronc_sup} deja renseigne' )

In [200]:
df_affectation.idtronc.unique()

3639

# SUITE

In [114]:
toto=list(at.recup_troncon_elementaire('TRONROUT0000000032862365',df,[]))

In [115]:
toto.append('TRONROUT0000000032862365')

In [ ]:
toto

In [117]:
at.recup_troncon_parallele_v2(df,toto)

C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees\agrege_troncon.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))


'TRONROUT0000000032876496'

In [ ]:
for i in toto :
    tata=at.affecter_troncon_ligne(i,df,df2_chaussees)
    print(len(tata))

In [12]:
dico=at.affecter_troncon(df,df2_chaussees)

0eme occurence : TRONROUT0000000084053027 à 18:23:53 nb ligne traite : 0, nb ligne differente=0
indexError ligne  TRONROUT0000000032954220
indexError ligne  TRONROUT0000000033016282
indexError ligne  TRONROUT0000000249471326
indexError ligne  TRONROUT0000000336640149
indexError ligne  TRONROUT0000000224966440
indexError ligne  TRONROUT0000000032833606
indexError ligne  TRONROUT0000000032895598
indexError ligne  TRONROUT0000000032991823
indexError ligne  TRONROUT0000000223846575
indexError ligne  TRONROUT0000000255628539
indexError ligne  TRONROUT0000000032895522
indexError ligne  TRONROUT0000000032843156
indexError ligne  TRONROUT0000000032966066
indexError ligne  TRONROUT0000000129676068
indexError ligne  TRONROUT0000000032855812
indexError ligne  TRONROUT0000000033029153
indexError ligne  TRONROUT0000000323460798
indexError ligne  TRONROUT0000000032937456
indexError ligne  TRONROUT0000000089679809
indexError ligne  TRONROUT0000000211249489
5000eme occurence : TRONROUT0000000032859682

In [13]:
with ct.ConnexionBdd('local_otv') as c :
    at.inserer_dico(c, dico)

POUR TRAITEMENT DES VOIES A 2 CHAUSSES : TROUVER PARRALLELE

In [87]:
#liste des troncons (parametre entrant)
liste_troncon=toto

In [88]:
#conversion en geodf
gdf_lignes=gp.GeoDataFrame(df.loc[liste_troncon], geometry='geom')
gdf_lignes.to_file(r'D:\temp\otv\17\2_chaussees\lignes_a_gouper.shp')

In [89]:
#union des geometries
gdf_lignes2=linemerge(gdf_lignes.unary_union)

In [ ]:
gdf_lignes2.wkt

In [90]:
#centroid de la ligne
xmin,ymin,xmax,ymax=gdf_lignes2.interpolate(0.5, normalized=True).buffer(50).bounds
point=gdf_lignes2.interpolate(0.5, normalized=True)

In [91]:
point.wkt

'POINT (385035.1182861876 6561685.156661048)'

In [92]:
#recherche des lignes proches du centroid
lignes_possibles=df.cx[xmin:xmax,ymin:ymax]
lignes_possibles.to_file(r'D:\temp\otv\17\2_chaussees\lignes_possibles.shp')

In [ ]:
lignes_possibles

In [94]:
#uniquement les lignes non présentes dans la liste de troncons avec le même nom de voie
ligne_filtres=lignes_possibles.loc[(~lignes_possibles.index.isin(liste_troncon)) & (lignes_possibles['numero']==gdf_lignes.iloc[0]['numero'])]

In [95]:
#obtenir les distances au centroid
ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
#garder uniquement la valeur la plus proche du centroid
ligne_proche=ligne_filtres.loc[ligne_filtres['distance']==ligne_filtres['distance'].min()].index.tolist()[0]
ligne_proche

'TRONROUT0000000129676313'

POUR TRAITEMENT DES ROND POINTS

In [4]:
#polygoniser les lignes
#creer la gdf
df.crs={'init':'epsg:2154'}

In [15]:
#creer la lsite des rd points selon le critere atmo aura ; creer une gdf avec la liste
liste_rd_points=[t.buffer(0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14] # car un cercle à un rappor de ce type entre 12 et 13
dico_rd_pt=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point=gp.GeoDataFrame(dico_rd_pt, geometry=liste_rd_points)
gdf_rd_point.crs={'init':'epsg:2154'}
gdf_rd_point.columns=['id_rdpt', 'facteur','geometry']
#on créer aussi la meme donnees avec un buffer interiuer, pour ne garder que les lignes dans le buffer exteriuer et hors buffer interieur (cas de rond point enjmabeant uine 2*2 et prenant la 2*2 voie qui est dans le polygone
liste_rd_points_int=[t.buffer(-0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14]
dico_rd_pt_int=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point_int=gp.GeoDataFrame(dico_rd_pt_int, geometry=liste_rd_points_int)
gdf_rd_point_int.crs={'init':'epsg:2154'}
gdf_rd_point_int.columns=['id_rdpt', 'facteur','geometry']

In [63]:
#jointure spataile pour une gdf avec uniquement les lignes des rd_points avec le numéro
l_dans_p=gp.sjoin(df,gdf_rd_point,op='within') 

In [64]:
#suprimer les doublons
l_dans_p_ss_doublon=l_dans_p.reset_index().rename(columns={'index':'id_ign'}).drop_duplicates('id_ign').set_index('id_ign')

In [66]:
l_dans_p_int=gp.sjoin(df,gdf_rd_point_int,op='within')

In [67]:
l_dans_p_final=l_dans_p_ss_doublon.loc[~l_dans_p_ss_doublon.index.isin(l_dans_p_int.index.tolist())]

In [68]:
#lignes qui touchent rd points
#1.ligne qui intersectent avec id_rdpt
l_intersct_rdpt=gp.sjoin(df,l_dans_p_final.drop('index_right', axis=1), how='inner',op='intersects')

In [69]:
#2.filtre de celle contenue dans le rd points 
l_intersct_rdpt=l_intersct_rdpt.loc[~l_intersct_rdpt.index.isin(l_dans_p_final.index.tolist())][['id_rdpt','numero_left','codevoie_d_left']]

In [70]:
#trouver le nb de voies qui intersectent chaque rd point et leur noms. renomer les colonnes
carac_rd_pt=(pd.concat([l_intersct_rdpt.groupby('id_rdpt').numero_left.nunique(),
    l_intersct_rdpt.groupby('id_rdpt')['numero_left'].apply(lambda x: ','.join(set(x))),
    l_intersct_rdpt.groupby('id_rdpt')['codevoie_d_left'].apply(lambda x: ','.join(set(x)))], axis=1))
carac_rd_pt.columns=['nb_rte_rdpt', 'nom_rte_rdpt','codevoie_rdpt']

In [ ]:
l_dans_p_final

In [297]:
#ajouter l'id_rdpt aux données
gdf_l_rd_point=pd.concat([df.drop('id_rdpt',axis=1),l_dans_p_final.loc[:,'id_rdpt']],axis=1, sort=False)

In [298]:
#mettre à jour la nature
gdf_l_rd_point['nature']=gdf_l_rd_point.apply(lambda x : 'Rd_pt' if x.id_rdpt>=0 else x['nature'], axis=1)

In [299]:
#ajouter les infos du rd point (nb voies différentes et nom)
gdf_avec_rd_pt=gdf_l_rd_point.merge(carac_rd_pt, how='left',left_on='id_rdpt', right_index=True)

In [320]:
def assigne_rdpt(id_rdpt,numero, liste_num, codevoie, liste_code) : 
    if id_rdpt >=0 :
        if (numero in liste_num) or (codevoie in liste_code) :
            return True
        else : 
            return False

In [321]:
gdf_avec_rd_pt['assigne_rdpt']=gdf_avec_rd_pt.apply(lambda x : assigne_rdpt(x['id_rdpt'],x['numero'], x['nom_rte_rdpt_y'], x['codevoie_d'],x['codevoie_rdpt']),axis=1)

In [ ]:
#gdf_avec_rd_pt.loc[gdf_avec_rd_pt.loc[:,'id_rdpt']==l_dans_p['id_rdpt']]
gdf_avec_rd_pt['id_rdpt']==l_dans_p['id_rdpt'].astype('float64')
#l_dans_p.loc[l_dans_p.loc[:,'id_rdpt']].sort_index().sort_index(axis=1)])

In [ ]:
#pour export et verif uniquement

#gdf_l_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test2.shp')
#gdf_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test1.shp')
#l_intersct_rdpt.to_file(r'E:\Boulot\otv\test_linearisation_auto\test3.shp')
df.to_file(r'E:\Boulot\otv\test_linearisation_auto\test7.shp')
#df.to_file(r'D:\temp\otv\17\test_rd_point\test5.shp')

In [91]:
','.join(set(['a','a','b','c']))

'b,c,a'

In [228]:
id_ign_ligne='TRONROUT0000000342862858'
df_ligne = df.loc[id_ign_ligne]
ligne_traite_troncon=[]
ligne_traite_troncon.append(id_ign_ligne)
liste_ligne_suivantes=[] 
liste_ligne_touchees=[]

In [ ]:
for key, value in {'nb_intrsct_src':['source', 'src_geom'],'nb_intrsct_tgt':['target', 'tgt_geom']}.items() :
    df_touches_source = df.loc[(~df.index.isin(ligne_traite_troncon)) & ((df['source'] == df_ligne[value[0]]) | (df['target'] == df_ligne[value[0]]))]
    df_rd_pt=df.loc[df['id_rdpt']==df_touches_source.iloc[0]['id_rdpt']]
    l_df_rd_pt=df.loc[df.index.isin(gp.sjoin(df,df_rd_pt, how='inner',op='intersects').index.unique().tolist())]
    l_df_rd_pt2=l_df_rd_pt.loc[~l_df_rd_pt.index.isin(df_rd_pt.index.tolist())].index.unique().tolist()
    l_df_rd_pt3=df.loc[df.index.isin(l_df_rd_pt2),'codevoie_d'].unique().tolist()
    print(l_df_rd_pt3)
    #liste_ligne_touchees+=df_touches_source.index.tolist()

In [233]:
df_touches_source['id_rdpt'].values.tolist()

[nan]

# TEST

In [138]:
%%time
a=np.array([i for i in range(100000)])
c=[]
for i,b in enumerate(a) :
    if b not in a :
        continue
    c=[b,b+1]
    a=a[np.isin(a,c, invert=True)]
    if i%50000==0 : print(i)

AttributeError: module 'numpy' has no attribute 'aray'

In [137]:
%%time
a=[i for i in range(100000)]
c=[]
for i,b in enumerate(a) :
    if b not in a :
        continue
    c=[b,b+1]
    a=[x for x in a if x not in c]
    if i%50000==0 : print(i)

0
50000
Wall time: 5min 14s


In [420]:
lignes_traitees=np.array([],dtype='<U24')
'1' in lignes_traitees

False

In [416]:
lignes_traitees.dtype

dtype('<U24')